<a href="https://colab.research.google.com/github/fadillarizalul/thesis-alzheimer/blob/main/alzheimer_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set up dan inisialisasi

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "fadillarizalul" # username from the json file
os.environ['KAGGLE_KEY'] = "5421cf5f298fedfa046e9db30f9f8ec3" # key from the json file
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images # api copied from kaggle

 73% 25.0M/34.1M [00:00<00:00, 48.2MB/s]
100% 34.1M/34.1M [00:00<00:00, 52.1MB/s]


In [3]:
# !mkdir tourist55/alzheimers-dataset-4-class-of-images
# !unzip -qq alzheimers-dataset-4-class-of-images.zip -d alzheimers-dataset-4-class-of-images
# !ls alzheimers-dataset-4-class-of-images

In [4]:
# creating train and test path of dataset

train_dir = "/content/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
test_dir = "/content/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test"

# augmentasi dan split data

In [5]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest', 
    validation_split=0.2
)

In [7]:
test_datagen = ImageDataGenerator(
    rescale = 1./225, 
    validation_split=0.2
)

In [8]:
seed = 1

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    seed=seed,
    subset='training'
)
 
val_generator = test_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    seed=seed,
    subset='validation'
)

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.


# create model architecture

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout

model = Sequential()

# Layer CNN pertama
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(150,150,3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Layer CNN kedua
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Layer CNN ketiga
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 36, 36, 64)        0